In [2]:
pip install tensorflow numpy matplotlib opencv-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import tensorflow as tf
from tensorflow.keras import layers, applications, models, callbacks
import pathlib
import os

# ==========================================
# 1. SETUP & CONFIGURATION (Your Local Path)
# ==========================================
# This matches your screenshot path
DATA_DIR = r"C:\Users\Saeer Jan\Desktop\project\Images\Images\new"

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10  # You can increase this to 15 if you have a GPU

print(f"🚀 Project Started! Looking for data in: {DATA_DIR}")

# Check if path works
if not os.path.exists(DATA_DIR):
    print("❌ ERROR: Could not find the folder!")
    print(f"Please check if this path is correct: {DATA_DIR}")
    exit()

# ==========================================
# 2. HANDLE IMBALANCE (The Class Weights)
# ==========================================
data_dir = pathlib.Path(DATA_DIR)
count_frontal = len(list(data_dir.glob('frontal/*')))
count_profile = len(list(data_dir.glob('profile/*')))

# Fallback if folders are named differently (e.g. Capital letters)
if count_frontal == 0: count_frontal = len(list(data_dir.glob('Frontal/*')))
if count_profile == 0: count_profile = len(list(data_dir.glob('Profile/*')))

total_images = count_frontal + count_profile

print(f"📊 Stats: Frontal={count_frontal}, Profile={count_profile}, Total={total_images}")

if total_images == 0:
    print("❌ Error: No images found. Check folder structure.")
    exit()

# Calculate Weights
weight_0 = (1 / count_frontal) * (total_images / 2.0)
weight_1 = (1 / count_profile) * (total_images / 2.0)
class_weights = {0: weight_0, 1: weight_1}

print(f"⚖️ Balanced Weights: {class_weights}")

# ==========================================
# 3. LOAD DATA
# ==========================================
print("🔄 Loading Images...")
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='binary'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='binary'
)

# Optimization for Local PC
train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

# ==========================================
# 4. BUILD MODEL (ResNet50)
# ==========================================
print("🧠 Building ResNet50 Brain...")
base_model = applications.ResNet50(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

inputs = tf.keras.Input(shape=IMG_SIZE + (3,))
x = layers.RandomFlip("horizontal")(inputs) # Augmentation
x = applications.resnet50.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs, outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# ==========================================
# 5. TRAIN & SAVE
# ==========================================
# Callbacks for safety
checkpoint = callbacks.ModelCheckpoint(
    'best_face_model.keras',  # Using .keras format to avoid warnings
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

print("🔥 Starting Training (This may take time on a laptop)...")
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    class_weight=class_weights,
    callbacks=[checkpoint]
)

print("\n✅ DONE! Model saved as 'best_face_model.keras'")

🚀 Project Started! Looking for data in: C:\Users\Saeer Jan\Desktop\project\Images\Images\new
📊 Stats: Frontal=5010, Profile=2004, Total=7014
⚖️ Balanced Weights: {0: 0.7, 1: 1.7499999999999998}
🔄 Loading Images...
Found 7014 files belonging to 2 classes.
Using 5612 files for training.
Found 7014 files belonging to 2 classes.
Using 1402 files for validation.
🧠 Building ResNet50 Brain...
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 29s 0us/step
🔥 Starting Training (This may take time on a laptop)...
Epoch 1/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7464 - loss: 0.6050
Epoch 1: val_accuracy improved from None to 0.96291, saving model to best_face_model.keras
176/176 ━━━━━━━━━━━━━━━━━━━━ 435s 2s/step - accuracy: 0.8074 - loss: 0.4791 - val_accuracy: 0.9629 - val_loss: 0.2815
Epoch 2/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9260 - loss: 0.2814
Epoch 2: val_accuracy improved from 0.96291 to 0.98359, saving model to best_face_model.keras
176/176 ━━━━━━━━━━━━━━━━━━━━ 4

In [14]:
import tensorflow as tf
import numpy as np
import cv2  # OpenCV for reading images
import os

# 1. Load the model
MODEL_PATH = 'best_face_model.keras'

if not os.path.exists(MODEL_PATH):
    print("❌ Model not found! Please run train.py first.")
    exit()

print(f"🔄 Loading model: {MODEL_PATH}...")
model = tf.keras.models.load_model(MODEL_PATH)

# 2. Function to predict
def predict_face(image_path):
    if not os.path.exists(image_path):
        print(f"❌ Image not found: {image_path}")
        return

    # Load and resize the image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Fix colors
    img = cv2.resize(img, (224, 224))

    # Preprocess
    img_array = np.expand_dims(img, axis=0)
    img_array = tf.keras.applications.resnet50.preprocess_input(img_array)

    # Predict
    prediction = model.predict(img_array)
    score = prediction[0][0]

    print(f"\n📸 Analysis for: {image_path}")
    print("-" * 30)

    # 0 = Frontal, 1 = Profile (Based on your folder order)
    if score > 0.5:
        confidence = score * 100
        print(f"✅ RESULT: PROFILE (Side View)")
        print(f"📊 Confidence: {confidence:.2f}%")
    else:
        confidence = (1 - score) * 100
        print(f"✅ RESULT: FRONTAL (Front View)")
        print(f"📊 Confidence: {confidence:.2f}%")

# ==========================================
# CHANGE THIS TO TEST A NEW IMAGE
# ==========================================
# Put the path of a test image here:
test_image_path = r"C:\Users\Saeer Jan\Pictures\Camera Roll\WIN_20251214_18_25_58_Pro.jpg"

predict_face(test_image_path)

🔄 Loading model: best_face_model.keras...
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step

📸 Analysis for: C:\Users\Saeer Jan\Pictures\Camera Roll\WIN_20251214_18_25_58_Pro.jpg
------------------------------
✅ RESULT: FRONTAL (Front View)
📊 Confidence: 77.74%
